In [ ]:
'''
A Multilayer Perceptron implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

In [1]:
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 10 # 1st layer number of features #256 originally
n_hidden_2 = 10 # 2nd layer number of features #256 origianlly
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [3]:
# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [4]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

In [5]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))
    
    # Backup params
    params = []
    for v in tf.trainable_variables():
        params.append(v.eval(sess))

Epoch: 0001 cost= 13.709293215
Epoch: 0002 cost= 2.704497353
Epoch: 0003 cost= 2.083915645
Epoch: 0004 cost= 1.785042544
Epoch: 0005 cost= 1.573117600
Epoch: 0006 cost= 1.433317507
Epoch: 0007 cost= 1.307368351
Epoch: 0008 cost= 1.180201922
Epoch: 0009 cost= 1.082589110
Epoch: 0010 cost= 1.012994598
Epoch: 0011 cost= 0.963764011
Epoch: 0012 cost= 0.925884143
Epoch: 0013 cost= 0.893376676
Epoch: 0014 cost= 0.865300137
Epoch: 0015 cost= 0.840081028
Optimization Finished!
Accuracy: 0.6798


In [43]:
# JSONify data
import numpy as np
import json

def SimpleEncode(ndarray):
    return json.dumps(ndarray.tolist())
#def SimpleDecode(jsonDump):
#    return np.array(json.loads(jsonDump))

param_names = ["W1", "W1", "Wout", "b1", "b2", "bout"]

i = 0
for param_name in param_names:
    print(str(i) + ". Param \"" + param_name + "\"")
    json_string = SimpleEncode(params[i].astype(np.float32))
    print(params[i].shape)
    #%timeit SimpleDecode(json_string)
    with open(param_name + '.json', 'w') as outfile:
        outfile.write(json_string)
        outfile.close
    print(params[i][0])
    i = i + 1
    
# Test images
json_string = SimpleEncode(mnist.test.images[:100].astype(np.float32))
with open('test_images.json', 'w') as outfile:
    outfile.write(json_string)
    outfile.close
# Test labels
json_string = SimpleEncode(mnist.test.labels[:100].astype(np.float32))
with open('test_labels.json', 'w') as outfile:
    outfile.write(json_string)
    outfile.close

0. Param "W1"
(784, 10)
[  5.99414349e-01  -2.12141359e-03   9.74539518e-01  -1.94176380e-02
   1.93531132e+00   1.26779485e+00  -2.25854564e+00   1.06665805e-01
  -3.81657630e-02  -1.00182879e+00]
1. Param "W1"
(10, 10)
[-0.66702491  0.31501833  0.44565055 -0.38444865 -1.11233985  0.63043249
  1.01179647  0.32583073  0.5981034   0.17731838]
2. Param "Wout"
(10, 10)
[-0.24018395  0.29888403  0.52436441  1.38239908 -0.51079571  0.63783181
 -1.49108851 -0.45945495  0.88876003  0.57916498]
3. Param "b1"
(10,)
1.3298
4. Param "b2"
(10,)
-0.473598
5. Param "bout"
(10,)
-1.52874


In [45]:
# Store layers weight & bias

weights = {
    'h1': tf.Variable(params[0]),
    'h2': tf.Variable(params[1]),
    'out': tf.Variable(params[2]),
}
biases = {
    'b1': tf.Variable(params[3]),
    'b2': tf.Variable(params[4]),
    'out': tf.Variable(params[5])
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    # Produce outputs
    #or image in mnist.test.images[:100]:
    _, c = sess.run(multilayer_perceptron, feed_dict={x: mnist.test.images[:100]})

TypeError: Fetch argument <function multilayer_perceptron at 0x7ff1df3b62f0> of <function multilayer_perceptron at 0x7ff1df3b62f0> has invalid type <class 'function'>, must be a string or Tensor. (Can not convert a function into a Tensor or Operation.)